## 1. Import Librairies and  Load dataset

In [78]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
ticket_data_df = pd.read_csv("ticket_data.csv")
cities_df = pd.read_csv("cities.csv")
stations_df = pd.read_csv("stations.csv")
providers_df = pd.read_csv("providers.csv")

## 2. Data cleaning and preparation

In [79]:
DATA_FORMAT = '%Y%m%d %H:%M:%S.%f'
ticket_data_df = ticket_data_df.replace(np.nan, '', regex=True)
cities_df = cities_df.replace(np.nan, '', regex=True)
stations_df = stations_df.replace(np.nan, '', regex=True)
providers_df = providers_df.replace(np.nan, '', regex=True)

ticket_data_df['price_in_cents'] =  ticket_data_df['price_in_cents'] / 100
ticket_data_df = ticket_data_df.rename(columns={'price_in_cents': 'prices'})

ticket_data_df['departure_ts'] =  pd.to_datetime(ticket_data_df['departure_ts'], format=DATA_FORMAT)
ticket_data_df['arrival_ts'] =  pd.to_datetime(ticket_data_df['arrival_ts'], format=DATA_FORMAT)
ticket_data_df['search_ts'] =  pd.to_datetime(ticket_data_df['search_ts'], format=DATA_FORMAT)

## 3. Extract essential informations ( MIN , MAX , AVG )

In [80]:
def get_hours_diff(row):
   diff = (row["arrival_ts"]-row["departure_ts"])
   return diff
# Price MIN, MIN , AVG
min_price = ticket_data_df['prices'].min()
max_price = ticket_data_df['prices'].max()
avg_price = ticket_data_df['prices'].mean()
print("## PRICE ##")
print("The min price by travel is : "+str(min_price)+" €")
print("The max price by travel is : "+str(max_price)+" €")
print("The average price by travel is : "+str(avg_price)+" €")
print("## ENDPRICE ##\n")
# Duration MIN, MIN , AVG
ticket_data_df['duration'] = ticket_data_df.apply(lambda x : x["arrival_ts"]-x["departure_ts"],axis=1)
ticket_data_df['duration_in_seconds'] = ticket_data_df.apply(lambda x : x['duration'].total_seconds(),axis=1)
min_duration = ticket_data_df['duration'].min()
max_duration = ticket_data_df['duration'].max()
avg_duration = ticket_data_df['duration'].mean()
print("## DURATION ##")
print("The min duration by travel is : "+str(min_duration)+" hours")
print("The max duration by travel is : "+str(max_duration)+" hours")
print("The average duration by travel is : "+str(avg_duration)+" hours")
print("## ENDDURATION ##\n")

## PRICE ##
The min price by travel is : 3.0 €
The max price by travel is : 385.5 €
The average price by travel is : 43.82711061373479 €
## ENDPRICE ##

## DURATION ##
The min duration by travel is : 0 days 00:20:00 hours
The max duration by travel is : 20 days 12:51:00 hours
The average duration by travel is : 0 days 07:04:37.247600043 hours
## ENDDURATION ##



##4. Get information by groups

In [81]:
# Join others dataset , to retreive informations quickly
ticket_data_df = pd.merge(ticket_data_df, cities_df[["id","latitude","longitude"]], how='left',
                          left_on='o_city', right_on='id', suffixes=('', 'o_city') )

del ticket_data_df['ido_city']
ticket_data_df = ticket_data_df.rename(columns={'latitude': 'latitude_o_city', 'longitude': 'longitude_o_city'})

ticket_data_df = pd.merge(ticket_data_df, cities_df[["id","latitude","longitude"]], how='left',
                          left_on='d_city', right_on='id', suffixes=('', 'd_city') )
del ticket_data_df['idd_city']
ticket_data_df = ticket_data_df.rename(columns={'latitude': 'latitude_d_city', 'longitude': 'longitude_d_city'})

ticket_data_df = pd.merge(ticket_data_df, providers_df[["id","name","transport_type"]], how='left',
                          left_on='company', right_on='id', suffixes=('', '_'))

del ticket_data_df['id_']

In [82]:
#compute distancies and difines groups
def get_distancies(row):
    try:
      return geodesic((row["latitude_d_city"],row["longitude_d_city"]),
                      (row["latitude_o_city"],row["longitude_o_city"])).km
    except:
      return 0

ticket_data_df['distancies'] = ticket_data_df.apply(get_distancies,axis=1)

def set_distancies_group(row):
    if row["distancies"] >  2000:
      return "2000+km"
    elif row["distancies"] >  800 and row["distancies"] <=  2000:
      return "800-2000km"
    elif row["distancies"] >=  201 and row["distancies"] <=  800 :
      return "201-800km"
    elif row["distancies"] >=  0 and row["distancies"] <=  200:
      return "0-200km"

ticket_data_df['distancies_groups'] = ticket_data_df.apply(set_distancies_group,axis=1)


In [83]:
#set transport type
def get_id_from_string(str) :
  return (str.replace('{','').replace('}','')).split(",")

def get_convoy_by_provider_id(id) :
    return (providers_df.loc[providers_df['id'] == int(id)]["transport_type"]).tolist()[0]

def get_convoy(row):
    convoy = row["transport_type"]
    if row["other_companies"] :
      mid_companies = get_id_from_string(row["other_companies"])
      if mid_companies :
          for id in mid_companies :
              c = get_convoy_by_provider_id(id)
              if c not in convoy :
                convoy = convoy + "-" + c
    return convoy
  
ticket_data_df['convoy'] = ticket_data_df.apply(get_convoy,axis=1)


In [84]:
d_max = ticket_data_df.groupby(["distancies_groups", "convoy"])["duration_in_seconds"].max().reset_index()
d_max["duration"] = pd.to_timedelta(d_max["duration_in_seconds"], unit='S')
#del d_max["duration_in_seconds"]

d_min = ticket_data_df.groupby(["distancies_groups", "convoy"])["duration_in_seconds"].min().reset_index()
d_min["duration"] = pd.to_timedelta(d_min["duration_in_seconds"], unit='S')
#del d_min["duration_in_seconds"]

d_mean = ticket_data_df.groupby(["distancies_groups", "convoy"])["duration_in_seconds"].mean().reset_index()
d_mean["duration"] = pd.to_timedelta(d_mean["duration_in_seconds"], unit='S')
#del d_mean["duration_in_seconds"]

p_max = ticket_data_df.groupby(["distancies_groups", "convoy"])["prices"].max().reset_index()
p_min = ticket_data_df.groupby(["distancies_groups", "convoy"])["prices"].min().reset_index()
p_mean = ticket_data_df.groupby(["distancies_groups", "convoy"])["prices"].mean().reset_index()


In [85]:
print("#price grouped by min,max,mean")
print("-max")
print(p_max)
print("-min")
print(p_min)
print("-mean")
print(p_mean)
print("#Duration grouped by min,max,mean")
print("-max")
print(d_max)
print("-min")
print(d_min)
print("-mean")
print(d_mean)

#if needed, we use to_csv() to export dataframe as csv

#price grouped by min,max,mean
-max
   distancies_groups      convoy  prices
0            0-200km         bus  229.00
1            0-200km   bus-train   63.30
2            0-200km  carpooling  128.50
3            0-200km       train  251.00
4            0-200km   train-bus   57.10
5          201-800km         bus  145.96
6          201-800km   bus-train  224.80
7          201-800km  carpooling  138.00
8          201-800km       train  385.50
9          201-800km   train-bus  164.50
10        800-2000km         bus  174.00
11        800-2000km  carpooling  161.50
12        800-2000km       train  375.50
-min
   distancies_groups      convoy  prices
0            0-200km         bus    8.50
1            0-200km   bus-train    9.90
2            0-200km  carpooling    3.00
3            0-200km       train    4.90
4            0-200km   train-bus   15.10
5          201-800km         bus   10.00
6          201-800km   bus-train   23.50
7          201-800km  carpooling    8.50
8          201-8